# Visualizando a COVID-19 no Brasil
> Evolução e estágio atual dos estados brasileiros

- toc: true 
- badges: true
- hide_binder_badge: true
- comments: true
- categories: [colab]
- image: images/br-maps.png

> Caso esteja visualizando este post em um dispositivo móvel, por favor utilize seu dispositivo na horizontal.

Este post reúne análises das ocorrências de COVID-19 nos estados do Brasil. Os dados utilizados para análise foram coletados pelo projeto [Brasil.io](https://brasil.io) dos sites das secretarias de saúde dos estados e municípios brasileiros.

A análise inicial apresenta diferentes métricas, tanto absolutas (confirmados e mortes) quanto relativas (confirmados e mortes por 100 mil habitantes). Além disso, apresentamos a análise por dia e em dias decorridos desde o primeiro caso.

> Sempre que alternar entre tipos de condição (confirmado ou morte), clique duas vezes na área principal do gráfico para que o zoom se ajuste (a área principal do gráfico tem pano de fundo quadriculado).

A análise final se concentra em casos confirmados e usa georeferenciamento para visualizar a trajetória do vírus pelo Brasil. No mapa, é possível visualizar informações sobre cada localidade passando o mouse por cima do estado desejado.

> É possível animar a linha do tempo da evolução dos casos apertando o botão play, ou visualizar uma data específica usando o slider.

Em cada gráfico, várias outras opções de interação estão disponíveis, podendo ser exploradas pelo menu que aparece no canto superior direito de cada um.

In [ ]:
#hide
!pip install -U plotly 

In [ ]:
#hide
import requests
import datetime

import folium
import pandas as pd
import numpy as np
import plotly.express as px

#### Evolução e estágio atual

In [ ]:
#hide
page = 1
base_url = url = "https://brasil.io/api/dataset/covid19/caso/data?format=json&page=" 
df_list = []

while True:
    json_brazil = requests.get(f"{url}{page}").json()
    if "count" not in json_brazil:
        break
    df_list.append(pd.DataFrame(json_brazil["results"]))
    page += 1

df = pd.concat(df_list)

In [ ]:
#hide
def reindex_by_date(df, max_day):
    min_day = min(df["date"])
    df = df.set_index("date")
    dates = pd.date_range(min_day, max_day, name="date")
    return df.reindex(dates).ffill()

In [ ]:
#hide
df_states = df.query("place_type == 'state'")
df_states = df_states.drop(["city","is_last","order_for_place","place_type"], axis=1)

last_day = max(df_states["date"])
df_states["date"] = pd.to_datetime(df_states["date"])
df_states = df_states.groupby("state").apply(lambda df: reindex_by_date(df, last_day))
df_states = df_states.reset_index("date").sort_values("date")

df_states.columns = ["Data", "Código IBGE", "Confirmado", "Confirmado / cem mil", "Letalidade", "Morte", "População", "UF"]
df_states["Morte / cem mil"] = df_states["Morte"] / df_states["População"] * 100000
df_states = df_states.drop(["Letalidade", "População"], axis=1)

ts_states = df_states.melt(id_vars=["Código IBGE", "Data", "UF"], var_name="Métrica", value_name="Quantidade")

##### Acumulado de casos por dia

In [ ]:
#hide_input
po = px.line(ts_states, x="Data", y="Quantidade", color="UF", animation_frame="Métrica")
po.update_layout(
#    title='Evolução dos casos de COVID-19 no Brasil',
    xaxis_title="Dia",
    yaxis_title="Métrica",
    updatemenus=[{"visible": False}],
)

##### Acumulado de casos em dias desde o primeiro caso confirmado

In [ ]:
#hide
df_states["Data"] = pd.to_datetime(df_states["Data"])
start_date = df_states.groupby("UF")["Data"].min().reset_index()
start_date.columns = ["UF", "Dia0"]

# adding day zero to df
df_day0 = pd.merge(start_date, df_states)
days_elapsed = pd.to_datetime(df_day0["Data"]) - pd.to_datetime(df_day0["Dia0"])
df_day0["Dias"] = days_elapsed.astype('timedelta64[D]')

ts_day0 = df_day0.drop(["Código IBGE", "Data", "Dia0"], axis=1).melt(id_vars=["UF", "Dias"], var_name="Métrica", value_name="Quantidade")

In [ ]:
#hide_input
po = px.line(ts_day0, x="Dias", y="Quantidade", color="UF", animation_frame="Métrica")
po.update_layout(
#    title='Evolução dos casos de COVID-19 no Brasil',
    xaxis_title="Dias desde o primeiro caso confirmado",
    yaxis_title="Métrica",
    updatemenus=[{"visible": False}],
)

#### Trajetória de dispersão pelo Brasil

##### Acumulado de casos confirmados por UF ao longo do tempo

In [ ]:
#hide
df_states["log_confirmed"] = np.log1p(df_states["Confirmado / cem mil"])
df_states["log_deaths"] = np.log1p(df_states["Morte / cem mil"])

brazil = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/?formato=application/vnd.geo+json&resolucao=2").json()
states = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados").json()
for state in brazil['features']:
    state['id'] = state['properties']['codarea']

df_deaths = df_states.query("Morte > 0").sort_values(["Data"])
df_states["Data"] = df_states["Data"].dt.strftime("%d-%m")
df_deaths["Data"] = df_deaths["Data"].dt.strftime("%d-%m")